#### NLP | MVP

# Coronavirus Tweets: April 2020<a id='top'></a> 

# 

## **Analysis Goal**  
The client, [name](link), ...
wants to ... and 
requested a model to find a pattern ... in tweets in April. 

Impetus?

**RQ:** ....?

## **Process**
**Data source:** 
Coronavirus COVID-19 Tweets [early](https://www.kaggle.com/datasets/smid80/coronavirus-covid19-tweets-early-april) and [late](https://www.kaggle.com/datasets/smid80/coronavirus-covid19-tweets-late-april) April


**Models:** 
* [name] model, fit a [model name](#??):
    * eval metric
* [name] model, fit a [model name](#??):
    * eval metric


## **Preliminary Visualizations**
[Figure 1. Name](#??)<br/>
[Figure 1. Name](#??)<br/>

## **Preliminary Conclusions**
The [name] model ... showed....<br/>

The next step .... 

In [2]:
import glob 
import nltk
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import re

import seaborn as sns
import string
pd.set_option('display.max_colwidth', None)
%matplotlib inline
%config InlineBackend.figure_formats = ['retina']  # or svg
sns.set(context='notebook', style='whitegrid')

from itertools import cycle
from nltk.tokenize import sent_tokenize, word_tokenize, RegexpTokenizer
from nltk.util import ngrams
from sklearn import svm
from sklearn.decomposition import PCA, NMF
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import BernoulliNB, MultinomialNB
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer 


from cleantext import clean





## 1 | Dataset: Coronavirus Tweets ([early](https://www.kaggle.com/datasets/smid80/coronavirus-covid19-tweets-early-april) and [late](https://www.kaggle.com/datasets/smid80/coronavirus-covid19-tweets-late-april) April) <a id='1'></a>  

In [ ]:
# # import one csv file to see data
# single_df = pd.read_csv('20200401.csv')

In [ ]:
# single_df.head(2)

In [ ]:
# single_df.info()

In [ ]:
# keep only the following 7 columns on full data import 

#  2   created_at            591480 non-null  object 
#  3   screen_name           591480 non-null  object 
#  4   text                  591480 non-null  object 
#  13  country_code          26040 non-null   object 
#  18  account_lang          0 non-null       float64
#  20  verified              591480 non-null  bool   
#  21  lang                  591480 non-null  object 

In [ ]:
# # read list of April 2020 files, while ommitting columns above

# path = r'/Users/sandraparedes/Documents/GitHub/metis_dsml/05_nlp/g00-nlp-project/data' 
# all_files = glob.glob(path + "/*.csv")

# li = []

# for filename in all_files:
#     df = pd.read_csv(filename, index_col=None, header=0, usecols=[2,3,4,13,18,20,21])
#     li.append(df)
# df = pd.concat(li, axis=0, ignore_index=True)


In [ ]:
# df.head()

In [ ]:
# df.info()

## 2 | Exploratory Data Analysis<a id='2'></a>  

In [ ]:
# df.info()

In [ ]:
# df['country_code'].unique()

In [ ]:
# df['lang'].unique()

In [ ]:
# print('English entries:', (df[df["lang"] == 'en'].count())['lang'])

In [ ]:
# print('US entries:', (df[df["country_code"] == 'US'].count())['country_code'])

In [ ]:
# eng_us_df = df[(df['country_code'] == 'US') &
#                (df['lang'] == 'en')]
# eng_us_df.info()


In [ ]:
# eng_us_df.head(10)

In [ ]:
# eng_us_df['text'].head(10)

### English + US df: 138,789 entries

In [ ]:
# #save cleaned df as tweet_df

# tweet_df = eng_us_df 
# tweet_df.to_pickle('coronavirus_tweets_df.pkl')
# tweet_df.to_csv(r'//Users/sandraparedes/Documents/GitHub/metis_dsml/05_nlp/g00-nlp-project/data/coronavirus_tweets_df.csv', index=False)


In [33]:
# #read df in
df = pd.read_csv('coronavirus_tweets_df.csv', low_memory=False)
df.head(10)

,created_at,screen_name,text,country_code,account_lang,verified,lang
0,2020-04-06T00:00:05Z,WFMGINC,....#SUNDAYFUNDAY #coronavirus style #vino cheers 🍷 https://t.co/SrymChBkq2,US,NaN,False,en
1,2020-04-06T00:00:14Z,jpomietlasz,"This pandemic has confirmed my worst fears, most people don’t know how to make entertaining videos. #Covid_19 #SinceIveBeenQuarantined #AmericasUnfunniestVideos #WrestleMania #tonyaharding",US,NaN,False,en
2,2020-04-06T00:00:17Z,harold0719,Is this true? \nhttps://t.co/Xj2zdFe7ab\n#EcuadorEnEmergencia #Covid_19 #CoronavirusPandemic,US,NaN,False,en
3,2020-04-06T00:00:18Z,uche_blackstock,"Many us thought it was Wuhan province, but it could never be us. Then it was Italy, but it could never be us. Now it is here. One #NewYorker died every 12 minutes from #COVID19 over this weekend. Absolutely devastating.\n\nhttps://t.co/h5woMTfSCH",US,NaN,True,en
4,2020-04-06T00:00:18Z,BrentALang,Ah #coronavirus humor https://t.co/1yXAZXNIgz,US,NaN,True,en
5,2020-04-06T00:00:18Z,grantstern,Miami and South Florida in general are also staying home.\n\nNorth Florida thinks their immune to #COVID19. https://t.co/OiPfVqyMvC,US,NaN,True,en
6,2020-04-06T00:00:20Z,Thom_Cordeiro,How can President Trump be flip about a question about continuity of power and contracting the #coronavirus? “When I am around him I don’t breathe”? And we wonder why other Americans question and defy the quarantine. #CoronavirusTaskForce,US,NaN,False,en
7,2020-04-06T00:00:24Z,shelbalynn17,"This is our most desperate hour. Help us, Obi-Wan Kenobi. You're our only hope. #Covid_19 #StarWars #weneedaleader",US,NaN,False,en
8,2020-04-06T00:00:25Z,LheimIndustries,Face your fears....\n#business #love #coronavirus #covid19 #nyc #ny #la #California #zoom #happy #prayer https://t.co/idDV3bpkUr,US,NaN,False,en
9,2020-04-06T00:00:28Z,Elliot_Fuchs,"First Defense Nasal Screen coming up on @ABCSharkTank during the #Covid_19 outbreak and all of a sudden the guy doesn’t seem crazy.\n\nSharks redeemed themselves w/ mega-offers.\n\nPost Corona-airing, @mcuban @TheSharkDaymond @kevinolearytv is gonna make a killing. Replaces mask-need",US,NaN,False,en


In [3]:
df = df.drop(columns=['created_at', 'screen_name', 'country_code', 
                 'account_lang', 'verified', 'lang'])
df.head(2)

,text
0,....#SUNDAYFUNDAY #coronavirus style #vino cheers 🍷 https://t.co/SrymChBkq2
1,"This pandemic has confirmed my worst fears, most people don’t know how to make entertaining videos. #Covid_19 #SinceIveBeenQuarantined #AmericasUnfunniestVideos #WrestleMania #tonyaharding"


### 2a | [start here with nlp](http://localhost:8888/notebooks/Documents/GitHub/metis_dsml/05_nlp/g1-nlp-overview/NLP_Supervised_Learning.ipynb)<a id='2a'></a>  

### Remove numbers, capital letters, and punctuation

In [4]:
alphanumeric = lambda x: re.sub('\w*\d\w*',' ', str(x))
punc_lower = lambda x: re.sub('[%s]' % re.escape(string.punctuation), ' ', x.lower())
                          
df['text'] = df.text.map(alphanumeric).map(punc_lower)
df.head(2)



,text
0,sundayfunday coronavirus style vino cheers 🍷 https t co
1,this pandemic has confirmed my worst fears most people don’t know how to make entertaining videos sinceivebeenquarantined americasunfunniestvideos wrestlemania tonyaharding


In [5]:
# remove emojis
df = df.astype(str).apply(lambda x: x.str.encode('ascii', 'ignore').str.decode('ascii'))


In [6]:
df.head(2)

,text
0,sundayfunday coronavirus style vino cheers https t co
1,this pandemic has confirmed my worst fears most people dont know how to make entertaining videos sinceivebeenquarantined americasunfunniestvideos wrestlemania tonyaharding


In [ ]:
#save clean df

clean_covid_tweets_df = df 
clean_covid_tweets_df.to_pickle('clean_covid_tweets_df.pkl')
clean_covid_tweets_df.to_csv(r'//Users/sandraparedes/Documents/GitHub/metis_dsml/05_nlp/g00-nlp-project/clean_covid_tweets_df.csv', index=False)


In [ ]:
# #read df in
# df = pd.read_csv('clean_covid_tweets_df.csv', low_memory=False)
# df.head(2)

In [9]:
# Count Vectorizer with min/max

X = df.text

cv = CountVectorizer(stop_words='english', min_df=0.01, max_df=.95)

text_cv = cv.fit_transform(X)

text_df_cv = pd.DataFrame(text_cv.toarray(), columns=cv.get_feature_names_out())
text_df_cv


,america,americans,amp,april,away,best,better,business,california,care,...,watch,way,week,weeks,work,workers,working,world,year,york
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
138791,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
138792,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
138793,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
138794,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [30]:
# my_name_is_sandra = cv.transform(['week workers'])
# my_name_is_sandra.toarray()

In [31]:
# text_cv.toarray()

### 3 | [sentiment analysis](http://localhost:8888/notebooks/Documents/GitHub/metis_dsml/05_nlp/g1-nlp-overview/nlp_unsupervised_nlp_exercises7.ipynb) code blocks<a id='3'></a>  

In [ ]:
analyzer = SentimentIntensityAnalyzer() 
sentiment = analyzer.polarity_scores(df).get('compound')
print('compound', sentiment)

In [ ]:
df['score'] = df.text.map(analyzer.polarity_scores).map(lambda x: x.get('compound'))
df.head()


### 4 | [dim redux](http://localhost:8888/notebooks/Documents/GitHub/metis_dsml/05_nlp/g2-dimensionality-reduction/nlp_unsupervised_dimensionality_reduction_exercises3.ipynb) and [PCA](http://localhost:8888/notebooks/Documents/GitHub/metis_dsml/05_nlp/g2-dimensionality-reduction/PCA_digits.ipynb)<a id='4'></a>  

# bookmark Thurs evening
# kernel quits

In [18]:
pca_5 = PCA(n_components=5)
pca_5.fit(text_cv.toarray())
pcafeatures = pca_5.transform(text_cv.toarray())


In [19]:
pcafeatures = pca_5.transform(text_cv.toarray())

In [21]:
text_cv.toarray().shape

(138796, 146)

In [32]:
pca_5_components = pca_5.components_
pca_5_components.shape

(5, 146)

In [ ]:
pc5_positives = []
pc5_positives.append(text_df_cv1.columns[4])
pc5_positives.append(text_df_cv1.columns[5])
pc5_positives

In [ ]:
#TD-IDF

### 4 | [LSA + NMF](http://localhost:8888/notebooks/Documents/GitHub/metis_dsml/05_nlp/g3-topic-modeling/Topic_Modeling_LSA_NMF.ipynb) [LSA chapter ex](http://localhost:8888/notebooks/Documents/GitHub/metis_dsml/05_nlp/g3-topic-modeling/nlp_unsupervised_topic_modeling_exercises2.ipynb), [NMF exercise](http://localhost:8888/notebooks/Documents/GitHub/metis_dsml/05_nlp/g01-nlp-exercises/05-nmf/nmf_solution-1.ipynb) 

In [ ]:
# doc_term       V visible variables      input (corpus matrix)
# doc_topic      W weights                feature set
# topic_term     H hidden weights         coefficients

In [ ]:
# method 1
# define model, pass n_components
# define doc_topic_W, pass doc_term_V

nmf_5 = NMF(n_components=5)
doc_topic = nmf_5.fit_transform(doc_term)

In [ ]:
nmf_5.components_.shape

In [ ]:
# pass topic, n_terms, model, terms

def get_top_terms(topic, n_terms, nmf=nmf_5, terms=terms):
    # get the topic components (i.e., term weights)
    components = nmf.components_[topic, :]

    # get term indices, sorted (descending) by topic weights
    top_term_indices = components.argsort()[-n_terms:]
    
    # use the `terms` array to get the actual top terms
    top_terms = np.array(terms)[top_term_indices]
    
    return top_terms.tolist()

In [ ]:
get_top_terms(2, 5)

In [ ]:
# define the topics
# pass function above

topics = ['-'.join(get_top_terms(i, 3)) for i in range(5)]
topics

In [ ]:
# define the categories
categories = [d[:d.find('.')] for d in docs]

In [ ]:
set(categories)

In [ ]:
topics = ['politics', 'entertainment', 'sport', 'tech', 'business']

In [ ]:
# method 2
# define vectorizer
# define doc_term_V, pass corpus

vectorizer = CountVectorizer(stop_words = 'english')
doc_term = vectorizer.fit_transform(example)


In [ ]:
# pass doc_term_V, index, vectorizer

pd.DataFrame(doc_term.toarray(), 
             index=ex_label, 
             columns=vectorizer.get_feature_names()).head(10)

# ex_label = [e[:30]+"..." for e in example]

In [ ]:
# define model with n topics
# define doc_topic_W, pass model + doc_term_V

nmf_2 = NMF(2)
doc_topic = nmf_model.fit_transform(doc_term)

In [ ]:
# define topic_word, index, cols, pass model + components_ 

topic_word = pd.DataFrame(nmf_2.components_.round(3),
             index = ["component_1","component_2"],
             columns = vectorizer.get_feature_names())
topic_word

In [ ]:
# define function to display topics, pass model, feat_names
def display_topics(model, feature_names, no_top_words, topic_names=None):
    for ix, topic in enumerate(model.components_):
        if not topic_names or not topic_names[ix]:
            print("\nTopic ", ix)
        else:
            print("\nTopic: '",topic_names[ix],"'")
        print(", ".join([feature_names[i]
                        for i in topic.argsort()[:-no_top_words - 1:-1]]))

In [ ]:
# no_top_words = 10
# display_topics(model, feature_names, no_top_words)

display_topics(nmf_2, vectorizer.get_feature_names(), 10)

In [ ]:
# define topic_terms_H 
# pass doc_topic_W

H = pd.DataFrame(doc_topic.round(5),
             index = ex_label,
             columns = ["component_1","component_2" ])
H